# Importing Libraries

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks  # type: ignore

2024-12-19 23:21:43.225227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 23:21:43.236960: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 23:21:43.240367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 23:21:43.249234: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Suppress NUMA and other non-critical warnings

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Suppresses warnings and info messages
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"  # Avoids NUMA checks

# Custom callback to print device information after each epoch

In [3]:
class DeviceCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Checking the device details
        devices = tf.config.list_logical_devices("GPU")
        if devices:
            device_name = devices[0].name
            print(f"Epoch {epoch + 1} finished using: {device_name} (GPU)")
        else:
            print(f"Epoch {epoch + 1} finished using: CPU")

# Check if GPU is available

In [4]:
if tf.config.list_physical_devices("GPU"):
    print("Training on GPU...")
else:
    print("No GPU detected. Exiting...")
    exit()

Training on GPU...


I0000 00:00:1734643305.520827    2621 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734643305.526975    2621 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734643305.527015    2621 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Load the CIFAR-10 dataset (50,000 training and 10,000 test images)

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d958be074577803d12ecdefd02955f39262c83c16fe9348329d7fe0b5c001ce so we will re-download the data.
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 113s 1us/step


# Normalize the data to range [0, 1]

In [6]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build a simple CNN model

In [7]:
model = models.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

/home/ohms/miniconda3/envs/.tf217/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1734643421.825373    2621 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734643421.825449    2621 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734643421.825468    2621 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1734643421.990422    2621 

# Compile the model

In [8]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Train the model with the custom callback

In [9]:
model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=[DeviceCallback()],
)

Epoch 1/20


I0000 00:00:1734643424.411153    3066 service.cc:146] XLA service 0x7ff7ec005da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734643424.411209    3066 service.cc:154]   StreamExecutor device (0): Quadro RTX 5000 with Max-Q Design, Compute Capability 7.5
2024-12-19 23:23:44.447927: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-19 23:23:44.636950: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 49/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1198 - loss: 2.2860

I0000 00:00:1734643426.262135    3066 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3345 - loss: 1.7929Epoch 1 finished using: /device:GPU:0 (GPU)
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.3346 - loss: 1.7925 - val_accuracy: 0.5385 - val_loss: 1.2821
Epoch 2/20
771/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5619 - loss: 1.2373Epoch 2 finished using: /device:GPU:0 (GPU)
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5622 - loss: 1.2367 - val_accuracy: 0.5996 - val_loss: 1.1348
Epoch 3/20
779/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6227 - loss: 1.0679Epoch 3 finished using: /device:GPU:0 (GPU)
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6227 - loss: 1.0678 - val_accuracy: 0.6475 - val_loss: 1.0064
Epoch 4/20
777/782 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6638 - loss: 0.9593Epoch 4 finished using: /device:GPU:0 (GPU)
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6639 - loss: 0.9591 - val_accuracy: 0.6518 - val_loss: 0.9905
Epoch 5/20
780/782 ━━━━━━━━

# Evaluate the model

In [10]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc * 100:.2f}%")

313/313 - 1s - 4ms/step - accuracy: 0.6981 - loss: 1.0740
Test accuracy: 69.81%
